In [ ]:
# download the python file to download open images data
!wget https://raw.githubusercontent.com/openimages/dataset/master/downloader.py

In [1]:
import pandas as pd
import sys
import os
import cv2
input_txt = 'data_download.txt'

# set limit for train, test and validation set for images to be downloaded
train_limit = 200000
test_limit = 50000
val_limit = 50000

In [2]:
train_df = pd.read_csv('../data/open_images_v4/train-annotations-bbox.csv')
train_df_filtered = train_df.drop_duplicates('ImageID').reset_index(drop=True)
val_df = pd.read_csv('../data/open_images_v4/validation-annotations-bbox.csv')
val_df_filtered = val_df.drop_duplicates('ImageID').reset_index(drop=True)
test_df = pd.read_csv('../data/open_images_v4/test-annotations-bbox.csv')
test_df_filtered = test_df.drop_duplicates('ImageID').reset_index(drop=True)

# get class mapping 
class_df = pd.read_csv('../data/open_images_v4/class-descriptions-boxable.csv', header=None)
class_df.index = class_df[0]
class_df = class_df.drop(columns=[0])
class_map = class_df.to_dict()[1]

In [3]:
# update LabelName with class
test_df['LabelName'] = test_df['LabelName'].map(class_map)
val_df['LabelName'] = val_df['LabelName'].map(class_map)
train_df['LabelName'] = train_df['LabelName'].map(class_map)

In [ ]:
# create list of images to be downloaded
with open(f"../data/open_images_v4/{input_txt}", "w+") as f:
  for idx, row in train_df_filtered.iterrows():
      sys.stdout.write(str(idx) + '\r')
      sys.stdout.flush()
      f.write("train/"+row['ImageID']+"\r")
      if idx>train_limit:
        break
  for idx, row in val_df_filtered.iterrows():
      sys.stdout.write(str(idx) + '\r')
      sys.stdout.flush()
      f.write("validation/"+row['ImageID']+"\r")
      if idx>val_limit:
        break
  for idx, row in test_df_filtered.iterrows():
      sys.stdout.write(str(idx) + '\r')
      sys.stdout.flush()
      f.write("test/"+row['ImageID']+"\r")
      if idx>test_limit:
        break

In [ ]:
# TO DO:
# update file name of .txt here

# download images
!python downloader.py $"../data/open_images_v4/data_download.txt" --download_folder=$"../data/open_images_v4/images" --num_processes=5

In [4]:
# get train, test, val subset ImageIDs for which images are downloaded
train_images = []
test_images = []
val_images = []
with open(f'../data/open_images_v4/{input_txt}','r') as f1:
    for line in f1:
        line_split = line.strip().split('/')
        (dataset, ImageID) = line_split
        if dataset=='train':
            train_images.append(ImageID)
        elif dataset=='test':
            test_images.append(ImageID)
        else:
            val_images.append(ImageID)

In [6]:
# for training generate annotation file

# filter train_df for the images downloaded
train_df = train_df[train_df.ImageID.isin(train_images)]

print(train_df.shape)

with open("../data/open_images_v4/train_annotation.txt", "w+") as f:
  for idx, row in train_df.iterrows():
      sys.stdout.write(str(idx) + '\r')
      sys.stdout.flush()
      if row['ImageID'] in train_images:
        img = cv2.imread('../data/open_images_v4/images/' + row['ImageID']+".jpg")
        height, width = img.shape[:2]
        x1 = int(row['XMin'] * width)
        x2 = int(row['XMax'] * width)
        y1 = int(row['YMin'] * height)
        y2 = int(row['YMax'] * height)
        
        fileName = os.path.join("data/open_images_v4/images", row['ImageID']+".jpg")
        className = row['LabelName']
        f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + ',' + str(height) + ',' + str(width) + '\n')

In [5]:
# for test

# filter test_df for the images downloaded
test_df = test_df[test_df.ImageID.isin(test_images)]

print(test_df.shape)

with open("../data/open_images_v4/test_annotation.txt", "w+") as f:
  for idx, row in test_df.iterrows():
      sys.stdout.write(str(idx) + '\r')
      sys.stdout.flush()
      if row['ImageID'] in test_images:
        img = cv2.imread('../data/open_images_v4/images/' + row['ImageID']+".jpg")
        height, width = img.shape[:2]
        x1 = int(row['XMin'] * width)
        x2 = int(row['XMax'] * width)
        y1 = int(row['YMin'] * height)
        y2 = int(row['YMax'] * height)
        
        fileName = os.path.join("data/open_images_v4/images", row['ImageID']+".jpg")
        className = row['LabelName']
        f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + ',' + str(height) + ',' + str(width) + '\n')
        

(294033, 13)


In [6]:
# for validation

# filter train_df for the images downloaded
val_df = val_df[val_df.ImageID.isin(val_images)]

print(val_df.shape)

with open("../data/open_images_v4/val_annotation.txt", "w+") as f:
  for idx, row in val_df.iterrows():
      sys.stdout.write(str(idx) + '\r')
      sys.stdout.flush()
      if row['ImageID'] in val_images:
        img = cv2.imread('../data/open_images_v4/images/' + row['ImageID']+".jpg")
        height, width = img.shape[:2]
        x1 = int(row['XMin'] * width)
        x2 = int(row['XMax'] * width)
        y1 = int(row['YMin'] * height)
        y2 = int(row['YMax'] * height)
        
        fileName = os.path.join("data/open_images_v4/images", row['ImageID']+".jpg")
        className = row['LabelName']
        f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + ',' + str(height) + ',' + str(width) + '\n')

(204621, 13)


In [ ]:
from skimage import io
import matplotlib.pyplot as plt

# sample plots
def plot_bbox(img_id):
  bboxs = []
  img_url = f'../data/open_images_v4/images/{img_id}.jpg'
  img = io.imread(img_url)
  height, width, channel = img.shape
  bboxs = test_df[test_df['ImageID']==img_id]
  #print(bboxs)
  for index, row in bboxs.iterrows():
      xmin = row['XMin']
      xmax = row['XMax']
      ymin = row['YMin']
      ymax = row['YMax']
      xmin = int(xmin*width)
      xmax = int(xmax*width)
      ymin = int(ymin*height)
      ymax = int(ymax*height)
      label_name = row['LabelName']
      print(f"Coordinates: {xmin,ymin}, {xmax,ymax}", label_name)
      cv2.rectangle(img, (xmin,ymin), (xmax,ymax), (255,0,0), 3)
      font = cv2.FONT_HERSHEY_SIMPLEX
      cv2.putText(img, label_name, (xmin,ymin-10), font, 1, (0,0,255), 3)
  plt.figure(figsize=(10,10))
  plt.title('Image with Bounding Box')
  plt.imshow(img)
  plt.axis("off")
  plt.show()

In [ ]:
plot_bbox('000026e7ee790996')

In [ ]:
plot_bbox('000aa0b1c8fd5ddf')

In [ ]:
plot_bbox('0003d84e0165d630')